# Permutation Flow-Shop Scheduling Problem

In [1]:
import json
import time
from dataclasses import asdict, dataclass

import pandas as pd

from bnbprob.pafssp import CallbackBnB, PermFlowShop2MHalf
from bnbpy import configure_logfile


In [2]:
configure_logfile("pfssp-experiments-2mhalf.log", mode="w")

In [3]:
import gc

gc.disable()

In [4]:
class WSFlowShop(PermFlowShop2MHalf):
    def warmstart(self):
        jobs = self.sequence
        p = jobs[0].p
        m = len(p)
        n = len(jobs)
        return super().randomized_heur(n * n / m, 42)

In [5]:
@dataclass
class Experiment:
    name: str
    warmstart: int
    lb1_start: int
    lb5_start: int
    incumbent: int
    lower_bound: int
    execution_time: float
    gap: float
    nodes: int
    sequence: list[int]


def run_experiment(
    name: str, p: list[list[int]], timelimit: int = 30
) -> Experiment:
    # Initialization
    problem = WSFlowShop.from_p(p)
    bnb = CallbackBnB(rtol=0.0001, save_tree=False)
    # Lower bounds and warmstart
    lb1_start = problem.calc_lb_1m()
    lb5_start = problem.calc_lb_2m()
    warmstart = problem.warmstart().calc_lb_1m()
    # Solve
    start_time = time.time()
    sol = bnb.solve(problem, maxiter=100_000_000, timelimit=timelimit)
    execution_time = time.time() - start_time
    # Results
    sequence_jobs = sol.problem.sequence
    sequence = [job.j for job in sequence_jobs] if sequence_jobs else []
    experiment = Experiment(
        name,
        warmstart,
        lb1_start,
        lb5_start,
        bnb.ub,
        bnb.lb,
        execution_time,
        bnb.gap,
        bnb.explored,
        sequence,
    )
    gc.collect()
    time.sleep(0.2)
    return experiment

In [6]:
problems = (
    [f'ta{str(i).zfill(3)}' for i in range(1, 21, 1)]
    + [f'ta{str(i).zfill(3)}' for i in range(31, 51, 1)]
)

experiments = list[Experiment]()

# Scan directory for all instances
for prob in problems:
    with open(
        f'./../data/flow-shop/{prob}.json', mode='r', encoding='utf8'
    ) as f:
        p = json.load(f)
    experiment = run_experiment(prob, p, timelimit=3600 * 2)
    experiments.append(experiment)
    print(experiment)

Experiment(name='ta001', warmstart=1278, lb1_start=1232, lb5_start=1278, incumbent=1278.0, lower_bound=1278.0, execution_time=0.03466439247131348, gap=0.0, nodes=85, sequence=[16, 2, 14, 7, 18, 13, 10, 12, 8, 5, 4, 6, 17, 15, 0, 3, 1, 9, 19, 11])
Experiment(name='ta002', warmstart=1359, lb1_start=1290, lb5_start=1355, incumbent=1359.0, lower_bound=1359.0, execution_time=0.8610551357269287, gap=0.0, nodes=26019, sequence=[5, 9, 16, 6, 18, 19, 8, 17, 14, 11, 7, 0, 13, 2, 12, 4, 10, 15, 3, 1])
Experiment(name='ta003', warmstart=1087, lb1_start=1073, lb5_start=1073, incumbent=1081.0, lower_bound=1081.0, execution_time=0.041826725006103516, gap=0.0, nodes=242, sequence=[2, 14, 13, 3, 15, 10, 9, 19, 17, 11, 0, 6, 18, 5, 4, 12, 8, 7, 16, 1])
Experiment(name='ta004', warmstart=1293, lb1_start=1268, lb5_start=1268, incumbent=1293.0, lower_bound=1293.0, execution_time=0.038263797760009766, gap=0.0, nodes=16, sequence=[12, 16, 18, 8, 15, 10, 6, 4, 9, 11, 14, 0, 19, 1, 3, 7, 5, 13, 2, 17])
Experim

In [7]:
df = pd.DataFrame([asdict(e) for e in experiments])
df.to_csv('pfsp-experiments-tai.csv', index=False)